In [1]:
import sys
import os

parent_directory = os.path.abspath(os.path.join('..'))
module_path = os.path.join(parent_directory,'src')
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from downLoad import wikipedia_scrape
from tqdm.notebook import tqdm

import pandas as pd

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [3]:
ufo_data=wikipedia_scrape('UFO',links_count=15)

Refenece Links count 968


Wikipedia pages scraped: 11


In [5]:
print(f"DataFrame Shape {ufo_data.shape}")
ufo_data.head(5)

DataFrame Shape (11, 4)


,topic,content,url,categories
0,UFO,An unidentified flying object (UFO) is any per...,https://en.wikipedia.org/wiki/Unidentified_fly...,"[All Wikipedia articles in need of updating, A..."
1,UFO,A mass sighting of celestial phenomena or unid...,https://en.wikipedia.org/wiki/1561_celestial_p...,"[1561 in the Holy Roman Empire, 16th-century e..."
2,UFO,The 1976 Tehran UFO Incident was a radar and ...,https://en.wikipedia.org/wiki/1976_Tehran_UFO_...,"[1976 in Iran, 20th century in Tehran, Alleged..."
3,UFO,A series of mass sightings of celestial phenom...,https://en.wikipedia.org/wiki/1566_celestial_p...,"[1566 in Europe, 1566 in science, 16th century..."
4,UFO,"The 1951 Pont-Saint-Esprit mass poisoning, als...",https://en.wikipedia.org/wiki/1951_Pont-Saint-...,"[1951 health disasters, 1951 in France, CS1 Fr..."


In [10]:
tqdm.pandas(desc="sentence_count")
ufo_data['sentence_count'] = ufo_data['content'].progress_apply(lambda x: len(x.split('.')))
ufo_data.sort_values(by=['sentence_count'],  ascending=False,inplace=True,ignore_index=True)
ufo_data.head()

sentence_count:   0%|          | 0/11 [00:00<?, ?it/s]

,topic,content,url,categories,sentence_count
0,UFO,An unidentified flying object (UFO) is any per...,https://en.wikipedia.org/wiki/Unidentified_fly...,"[All Wikipedia articles in need of updating, A...",428
1,UFO,The 2012 phenomenon was a range of eschatologi...,https://en.wikipedia.org/wiki/2012_phenomenon,"[2012 hoaxes, 2012 phenomenon, AC with 0 eleme...",360
2,UFO,"On 19 October 1986, a Tupolev Tu-134 jetliner ...",https://en.wikipedia.org/wiki/1986_Mozambican_...,"[1986 in Mozambique, 1986 in South Africa, Acc...",225
3,UFO,"The 1952 Washington, D.C. UFO incident, also k...","https://en.wikipedia.org/wiki/1952_Washington,...","[1952 in Washington, D.C., Central Intelligenc...",205
4,UFO,"On 23 April 2007, Captain Ray Bowyer was flyin...",https://en.wikipedia.org/wiki/2007_Alderney_UF...,"[2007 in Guernsey, Accidents and incidents inv...",117


In [32]:
input_doc= ufo_data['content'][0]
input_doc

'An unidentified flying object (UFO) is any perceived aerial phenomenon that cannot be immediately identified or explained. On investigation, most UFOs are identified as known objects or atmospheric phenomena, while a small number remain unexplained. \nScientists and skeptic organizations such as the Committee for Skeptical Inquiry have provided prosaic explanations for a large number of claimed UFOs being caused by natural phenomena, human technology, delusions, or hoaxes. Small but vocal groups of "ufologists" favour unconventional, pseudoscientific hypotheses, some of which go beyond the typical extraterrestrial visitation claims and sometimes form part of new religions.\nWhile unusual sightings have been reported in the sky throughout history, UFOs did not achieve their current cultural prominence until the period after World War II, escalating during the Space Age. The 20th century saw studies and investigations into UFO reports conducted by governments (such as Projects Grudge an

In [33]:
import pandas as pd
import re
import spacy
import neuralcoref
import networkx as nx
import matplotlib.pyplot as plt


nlp = spacy.load('en_core_web_lg')
neuralcoref.add_to_pipe(nlp)

In [14]:
text = re.sub(r'\n+', '.', input_doc)  # replace multiple newlines with period
text = re.sub(r'\[\d+\]', ' ', text)  # remove reference numbers
text = nlp(text)
text = nlp(text._.coref_resolved) 
text

An unidentified flying object (UFO) is any perceived aerial phenomenon that cannot be immediately identified or explained. On investigation, most UFOs are identified as known objects or atmospheric phenomena, while a small number remain unexplained. .Scientists and skeptic organizations such as the Committee for Skeptical Inquiry have provided prosaic explanations for a large number of claimed UFOs being caused by natural phenomena, human technology, delusions, or hoaxes. Small but vocal groups of "ufologists" favour unconventional, pseudoscientific hypotheses, some of which go beyond the typical extraterrestrial visitation claims and sometimes form part of new religions..While unusual sightings have been reported in the sky throughout history, UFOs did not achieve UFOs current cultural prominence until the period after World War II, escalating during the Space Age. The 20th century saw studies and investigations into UFO reports conducted by governments (such as Projects Grudge and Si

In [15]:
sentences = [sent.string.strip() for sent in text.sents] 
sentences

['An unidentified flying object (UFO) is any perceived aerial phenomenon that cannot be immediately identified or explained.',
 'On investigation, most UFOs are identified as known objects or atmospheric phenomena, while a small number remain unexplained.',
 '.Scientists and skeptic organizations such as the Committee for Skeptical Inquiry have provided prosaic explanations for a large number of claimed UFOs being caused by natural phenomena, human technology, delusions, or hoaxes.',
 'Small but vocal groups of "ufologists" favour unconventional, pseudoscientific hypotheses,',
 'some of which go beyond the typical extraterrestrial visitation claims and sometimes form part of new religions..',
 'While unusual sightings have been reported in the sky throughout history, UFOs did not achieve UFOs current cultural prominence until the period after World War II, escalating during the Space Age.',
 'The 20th century saw studies and investigations into UFO reports conducted by governments (suc

In [34]:
def get_entity_pairs(text, coref=True):
    # preprocess text
    text = re.sub(r'\n+', '.', text)  # replace multiple newlines with period
    text = re.sub(r'\[\d+\]', ' ', text)  # remove reference numbers
    text = nlp(text)
    if coref:
        text = nlp(text._.coref_resolved)  # resolve coreference clusters

    def refine_ent(ent, sent):
        unwanted_tokens = (
            'PRON',  # pronouns
            'PART',  # particle
            'DET',  # determiner
            'SCONJ',  # subordinating conjunction
            'PUNCT',  # punctuation
            'SYM',  # symbol
            'X',  # other
        )
        ent_type = ent.ent_type_  # get entity type
        if ent_type == '':
            ent_type = 'NOUN_CHUNK'
            ent = ' '.join(str(t.text) for t in
                           nlp(str(ent)) if t.pos_
                           not in unwanted_tokens and t.is_stop == False)
        elif ent_type in ('NOMINAL', 'CARDINAL', 'ORDINAL') and str(ent).find(' ') == -1:
            refined = ''
            for i in range(len(sent) - ent.i):
                if ent.nbor(i).pos_ not in ('VERB', 'PUNCT'):
                    refined += ' ' + str(ent.nbor(i))
                else:
                    ent = refined.strip()
                    break

        return ent, ent_type

    sentences = [sent.string.strip() for sent in text.sents]  # split text into sentences
    ent_pairs = []
    for sent in sentences:
        sent = nlp(sent)
        spans = list(sent.ents) + list(sent.noun_chunks)  # collect nodes
        spans = spacy.util.filter_spans(spans)
        with sent.retokenize() as retokenizer:
            [retokenizer.merge(span, attrs={'tag': span.root.tag,
                                            'dep': span.root.dep}) for span in spans]
        deps = [token.dep_ for token in sent]

        # limit our example to simple sentences with one subject and object
        if (deps.count('obj') + deps.count('dobj')) != 1\
                or (deps.count('subj') + deps.count('nsubj')) != 1:
            continue

        for token in sent:
            if token.dep_ not in ('obj', 'dobj'):  # identify object nodes
                continue
            subject = [w for w in token.head.lefts if w.dep_
                       in ('subj', 'nsubj')]  # identify subject nodes
            if subject:
                subject = subject[0]
                # identify relationship by root dependency
                relation = [w for w in token.ancestors if w.dep_ == 'ROOT']
                if relation:
                    relation = relation[0]
                    # add adposition or particle to relationship
                    if relation.nbor(1).pos_ in ('ADP', 'PART'):
                        relation = ' '.join((str(relation), str(relation.nbor(1))))
                else:
                    relation = 'unknown'

                subject, subject_type = refine_ent(subject, sent)
                token, object_type = refine_ent(token, sent)

                ent_pairs.append([str(subject), str(relation), str(token),
                                  str(subject_type), str(object_type)])

    ent_pairs = [sublist for sublist in ent_pairs
                          if not any(str(ent) == '' for ent in sublist)]
    pairs = pd.DataFrame(ent_pairs, columns=['subject', 'relation', 'object',
                                             'subject_type', 'object_type'])
    print('Entity pairs extracted:', str(len(ent_pairs)))

    return pairs

In [35]:
p=get_entity_pairs(input_doc)
p

Entity pairs extracted: 39


,subject,relation,object,subject_type,object_type
0,Small vocal groups,favour,unconventional pseudoscientific hypotheses,NOUN_CHUNK,NOUN_CHUNK
1,UFOs,achieve,UFOs current cultural prominence,NOUN_CHUNK,NOUN_CHUNK
2,The 20th century,saw,studies,DATE,NOUN_CHUNK
3,image,represents,secondhand depiction,NOUN_CHUNK,NOUN_CHUNK
4,astronomer J. Allen Hynek,conducted,small survey,PERSON,NOUN_CHUNK
5,Five,reported,UFO sightings,CARDINAL,NOUN_CHUNK
6,astrophysicist Peter A. Sturrock,conducted,large surveys,PERSON,NOUN_CHUNK
7,astronomer,headed,United States Air Force investigation,NOUN_CHUNK,NOUN_CHUNK
8,LaPaz,reported,personal sightings,ORG,NOUN_CHUNK
9,George Adamski,made,claims,PERSON,NOUN_CHUNK
